# **NOTEBOOK TEMPORAIRE POUR CALCUL DES DONNEES POP, EMPLOIS ET VEH_KM TOTAL**

In [1]:
%load_ext autoreload
%autoreload 2
import sys #c'est pas propre mais pour le moment pour importer mes modules perso dans le notebook je ne sais pas faire
sys.path.append(r'C:\Users\martin.schoreisz\git\Outils\Outils\Martin_Perso')
sys.path.append(r'C:\Users\martin.schoreisz\git\otv\otv\Transfert_Donnees')
sys.path.append(r'C:\Users\martin.schoreisz\git\Lin_uniq_BdxM\Lin_uniq_BdxM\Traitements')
import Connexion_Transfert as ct
import geopandas as gp
import pandas as pd
import numpy as np

In [3]:
deplacements=pd.read_excel(r'C:\Users\martin.schoreisz\Documents\AffairesEnCours\Linearisation_BM\C19SA0101\Doc_travail\Donnees_source\EMC\Linéarisation BdxM et EMC²\edaBx3.xls',sheet_name='3Dep')
trajet=pd.read_excel(r'C:\Users\martin.schoreisz\Documents\AffairesEnCours\Linearisation_BM\C19SA0101\Doc_travail\Donnees_source\EMC\Linéarisation BdxM et EMC²\edaBx3.xls',sheet_name='4Tra')
personnes=pd.read_excel(r'C:\Users\martin.schoreisz\Documents\AffairesEnCours\Linearisation_BM\C19SA0101\Doc_travail\Donnees_source\EMC\Linéarisation BdxM et EMC²\edaBx3.xls',sheet_name='2Per')

In [4]:
#liées les données sur l'index deplacement
jointure_totale=trajet.merge(deplacements[['D2A', 'D5A', 'IndexD', 'DOIB']], on ='IndexD')
jointure_totale.head(1)

,TP1,SEC,ZF,TP2,ECH,PER,NDEP,T1,T3,T8,T8B,IndexT,IndexD,IndexP,IndexM,COEQ,ZZZT,D2A,D5A,DOIB
0,4,1,1,1001,1,1,1,1,21,1.0,NaN,1,1,1,1,122.5547,1,1,11,27957


# Données de déplacement liées au domicile
le but est d'appliquer un coefficient a une population ramenée sur les lignes du rhv.  
on va donc  : 
- chercher tous les deplacements liés au domicile (aller et retour)
- calculer le nombre de trajet par personne enquetees, puis un nombre de trajets moyen sur ces personnes
- ramener ce nombre de trajets moyen à une situation plus générale en appliquant les coefficient de redressement de l'EMD

In [46]:
#filtrer les donnes selon le trajet en tant que conducteur d'un VL (T3==21), pour les deplacement a partir du domicile ou vers lui (D2A ou D5A ==1) ou les deplacement travail ((D2A ou D5A ==11))
filtre=jointure_totale.loc[((jointure_totale['D2A']==1) | (jointure_totale['D5A']==1)) & 
                   (jointure_totale['T3']==21)].sort_values(['SEC','IndexP']).copy()
#regrouper par secteur et personnes, en comptant le nb de déplacements
nb_depl_pers=filtre.groupby(['SEC', 'IndexP']).ZZZT.count().reset_index().rename(columns={'ZZZT':'nb_traj'})
#nombre de personnes enquetees par secteurs
nb_pers_enq_sect=personnes.groupby('SEC').IndexP.count().reset_index().rename(columns={'IndexP':'nb_pers'})
nb_traj_enq_sect=nb_depl_pers.groupby('SEC').nb_traj.sum().reset_index()

In [47]:
#jointure nombre de trajet / nombre de personnes puis calcul nb_trajets_moyen brut
nb_traj_nb_pers=nb_traj_enq_sect.merge(nb_pers_enq_sect, on ='SEC')
nb_traj_nb_pers['nb_traj_moy']=nb_traj_nb_pers.nb_traj/nb_traj_nb_pers.nb_pers
#redressement des données
#nb total de trajet pour les gens qui font les trajes recherchés
nb_depl_tot=nb_depl_pers.merge(personnes[['IndexP','COEQ']], on='IndexP')
nb_depl_tot['nb_traj_redres']=nb_depl_tot.nb_traj*nb_depl_tot.COEQ
nb_depl_tot_sect=nb_depl_tot.groupby('SEC').nb_traj_redres.sum().reset_index().rename(columns={'nb_traj_redres':'nb_traj_redres_tot'}).reset_index()
#nombre total de personnes
nb_pers_sect=personnes.groupby('SEC').COEQ.sum().reset_index().rename(columns={'COEQ':'nb_tot_pers'})
#nb moy de trajet en VL lie au domicile par pers
nb_traj_moy_redres=nb_depl_tot_sect.merge(nb_pers_sect, on='SEC')
nb_traj_moy_redres['nb_traj_moy']=nb_traj_moy_redres.nb_traj_redres_tot/nb_traj_moy_redres.nb_tot_pers
nb_traj_moy_redres.to_csv(r'D:\Boulot\AffairesEnCours\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_source\EMC\Linéarisation BdxM et EMC²\trajet_moy_travail_redress_coeq.csv')

In [48]:
nb_traj_moy_redres

,index,SEC,nb_traj_redres_tot,nb_tot_pers,nb_traj_moy
0,0,1,10942.29227,34594.87051,0.316298
1,1,2,34198.74410,52073.43830,0.656741
2,2,3,64917.96460,51656.14500,1.256733
3,3,4,29685.25280,53374.40240,0.556170
4,4,5,12439.31833,15091.80000,0.824244
5,5,6,18395.70031,22485.64078,0.818109
6,6,7,47659.89360,38052.01780,1.252493
7,7,8,93402.38190,59476.30960,1.570413
8,8,9,46788.03606,35865.40169,1.304545
9,9,10,34881.52570,38700.49200,0.901320


# Données de déplacement liées au travail
le but est d'appliquer un coefficient a un nombre d'emploi ramené sur les lignes du rhv.  
le pb c'est qu'il faut avoir un coefficient du nombre de déplacement en voiture par zone EDA pour aller au travail


In [171]:
#liées les données sur l'index deplacement, avec la nature du deplacement et les zones de depart et d'arrivees
jointure_totale=trajet.merge(deplacements[['D2A', 'D5A','D3S', 'D7S', 'IndexD']], on ='IndexD')
jointure_totale.head(1)

,TP1,SEC,ZF,TP2,ECH,PER,NDEP,T1,T3,T8,...,IndexT,IndexD,IndexP,IndexM,COEQ,ZZZT,D2A,D5A,D3S,D7S
0,4,1,1,1001,1,1,1,1,21,1.0,...,1,1,1,1,122.5547,1,1,11,1,333


In [172]:
#on veut conserver que les deplacements lies au travail, en voiture
filtre=jointure_totale.loc[(jointure_totale['D2A']==11) | (jointure_totale['D5A']==11)].sort_values(['SEC','IndexP']).copy()

In [173]:
#on ajoute l'attribiut qui decrit la zone, que ce soit de depart ou d'arrivée
filtre['SEC_emploi']=filtre.apply(lambda x : x['D3S'] if x['D2A']==11 else  x['D7S'], axis=1)
filtreMotorise=filtre.loc[filtre['T3']==21].copy()

In [174]:
#regroupe par personne (i.e par emploi) et on compte le nombre de deplacement
nbDeplacementMotorise=filtreMotorise.groupby(['SEC_emploi', 'IndexP']).ZZZT.count().reset_index().rename(columns={'ZZZT':'nb_traj_motorises'}).merge(personnes[['IndexP','COEQ']], on='IndexP', how='left')
nbDeplacementGlobal=filtre.groupby(['SEC_emploi', 'IndexP']).ZZZT.count().reset_index().rename(columns={'ZZZT':'nb_traj_tot'}).merge(personnes[['IndexP','COEQ']], on='IndexP', how='left')
nbDeplacementMotorise['nb_traj_motorises_equiv']=nbDeplacementMotorise.nb_traj_motorises*nbDeplacementMotorise.COEQ
nbDeplacementGlobal['nb_traj_tot_equiv']=nbDeplacementGlobal.nb_traj_tot*nbDeplacementGlobal.COEQ

In [175]:
#regroupement par secteur de l'emploi
nbDeplacementMotorisesSecteur=nbDeplacementMotorise.groupby('SEC_emploi').agg(nb_pers_motorisee=pd.NamedAgg(column='IndexP', aggfunc=lambda x :len(x)),
                                                nb_traj_motorises=pd.NamedAgg(column='nb_traj_motorises', aggfunc='sum'),
                                                nb_pers_motorises_equiv=pd.NamedAgg(column='COEQ', aggfunc='sum'),
                                                nb_traj_motorises_equiv=pd.NamedAgg(column='nb_traj_motorises_equiv', aggfunc='sum'))
nbDeplacementGlobalSecteur=nbDeplacementGlobal.groupby('SEC_emploi').agg(nb_pers_tot=pd.NamedAgg(column='IndexP', aggfunc=lambda x :len(x)),
                                                nb_traj_tot=pd.NamedAgg(column='nb_traj_tot', aggfunc='sum'),
                                                nb_pers_equiv=pd.NamedAgg(column='COEQ', aggfunc='sum'),
                                                nb_traj_tot_equiv=pd.NamedAgg(column='nb_traj_tot_equiv', aggfunc='sum'))

In [176]:
#calcul du ratio de trajet fait en voiture et du nombre moyen de trajet lie au travail par personne
NbTrajetEmploiMotoriseMoyen=nbDeplacementMotorisesSecteur.merge(nbDeplacementGlobalSecteur, on='SEC_emploi', how='left')
NbTrajetEmploiMotoriseMoyen['nb_moy_trajet_pers']=NbTrajetEmploiMotoriseMoyen.nb_traj_tot/NbTrajetEmploiMotoriseMoyen.nb_pers_tot
NbTrajetEmploiMotoriseMoyen['ratio_trajet_motorise']=NbTrajetEmploiMotoriseMoyen.nb_traj_motorises/NbTrajetEmploiMotoriseMoyen.nb_traj_tot
NbTrajetEmploiMotoriseMoyen['nb_moy_trajet_pers_equiv']=NbTrajetEmploiMotoriseMoyen.nb_traj_tot_equiv/NbTrajetEmploiMotoriseMoyen.nb_pers_equiv
NbTrajetEmploiMotoriseMoyen['ratio_trajet_motorise_equi']=NbTrajetEmploiMotoriseMoyen.nb_traj_motorises_equiv/NbTrajetEmploiMotoriseMoyen.nb_traj_tot_equiv

In [179]:
NbTrajetEmploiMotoriseMoyen.reindex(columns=['nb_pers_motorisee','nb_traj_motorises','nb_pers_tot','nb_traj_tot','nb_moy_trajet_pers','ratio_trajet_motorise','nb_pers_motorises_equiv',
                                             'nb_traj_motorises_equiv','nb_pers_equiv','nb_traj_tot_equiv','nb_moy_trajet_pers_equiv','ratio_trajet_motorise_equi']
                                   ).to_csv(r'D:\Boulot\AffairesEnCours\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\activite\moyenneDeplacementTravail.csv')

# Calcul d'un veh_km Total
**ATTENTION** : le veh km total calculé est basé sur une distance à vol d'oiseau, à partir de longueur de déplacement, dont une partie du trajet peut etre fait sans voiture. il faut donc prendre ce chiffre avec du recul. il ne sert que de comparaison avec les données calculées

In [14]:
#tous les trajets fait en voiture et les deplacements associés
listeDeplacementVoiture=jointure_totale.loc[jointure_totale.T3.isin((13,15,21,81))].IndexD.tolist()
#recuperation de ces deplacements et somme de la sitance enmetre pour chacun
jointure_totale.loc[jointure_totale.IndexD.isin(listeDeplacementVoiture),'veh_km_D']=jointure_totale.loc[jointure_totale.IndexD.isin(listeDeplacementVoiture)].COEQ*jointure_totale.loc[jointure_totale.IndexD.isin(listeDeplacementVoiture)].DOIB
Veh_km_total=jointure_totale.veh_km_D.sum()/1000

In [15]:
Veh_km_total

12257937.5473321